In [1]:
import urllib
import lxml
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re

In [2]:
url1 = 'https://en.wikipedia.org/wiki/Main_Page'
url2 = 'https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc'
url3 = 'https://www.imdb.com/india/top-rated-indian-movies/'
url4 = 'https://meesho.com/bags-ladies/pl/p7vbp'
url5a= 'https://www.icc-cricket.com/rankings/mens/team-rankings/odi'
url5b = 'https://www.icc-cricket.com/rankings/mens/player-rankings/odi/batting'
url5c= 'https://www.icc-cricket.com/rankings/mens/player-rankings/odi/bowling'
url7 = 'https://coreyms.com/'
url8 = 'https://www.nobroker.in/property/sale/bangalore/multiple?searchParam=W3sibGF0IjoxMi45OTgxNzMyLCJsb24iOjc3LjU1MzA0NDU5OTk5OTk5LCJwbGFjZUlkIjoiQ2hJSnhmVzREUE05cmpzUktzTlRHLTVwX1FRIiwicGxhY2VOYW1lIjoiUmFqYWppbmFnYXIiLCJzaG93TWFwIjpmYWxzZX0seyJsYXQiOjEyLjkzMDc3MzUsImxvbiI6NzcuNTgzODMwMiwicGxhY2VJZCI6IkNoSUoyZGRsWjVnVnJqc1JoMUJPQWFmLW9ycyIsInBsYWNlTmFtZSI6IkpheWFuYWdhciIsInNob3dNYXAiOmZhbHNlfSx7ImxhdCI6MTIuOTc4MzY5MiwibG9uIjo3Ny42NDA4MzU2LCJwbGFjZUlkIjoiQ2hJSmtRTjNHS1FXcmpzUk5oQlFKcmhHRDdVIiwicGxhY2VOYW1lIjoiSW5kaXJhbmFnYXIiLCJzaG93TWFwIjpmYWxzZX1d'
url10= 'https://www.bewakoof.com/women-tshirts?ga_q=tshirts'

In [3]:
def question1(url):
    text = urllib.request.urlopen(url).read().decode('utf-8')
    soup = bs(text)
    headers = soup.find_all('head')
    return headers

In [4]:
def question2(url):
    '''Since the top 100 are split over the two pages, two page links are created'''
    page1 = requests.get(url)
    page2 = requests.get(url + '&start=51&ref_=adv_nxt')
    movie_list = []
    for _ in [page1, page2]:
        soup = bs(_.content)
        text = soup.find_all('h3', {'class':'lister-item-header'})
        movie_list += [_.text.split('\n')[1:4] for _ in text]
    df = pd.DataFrame(movie_list, columns = ['Rank', 'Movie Name', 'Year'])
    df.set_index('Rank', inplace = True)
    return df

In [5]:
def question3(url):
    '''
    Since the imdb site list the top 250 movies, the list is truncated to 
    the top 100 as required
    '''
    page = requests.get(url)
    soup = bs(page.content)
    text = soup.find_all('td', {'class':'titleColumn'})
    rate = soup.find_all('td', {'class':'ratingColumn imdbRating'})
    names  = [_.text.split('\n')[1:4] for _ in text][0:100] 
    rating = [_.text.split('\n')[1] for _ in rate][0:100]

    df = pd.DataFrame(names, columns = ['Rank', 'Movie Name', 'Year'])
    df.set_index('Rank', inplace = True)
    df['rating'] = rating
    return df

In [6]:
def question4(url):
    page = requests.get(url)
    text = soup(page.content)
    name = soup.find_all('p', {'class':'Text__StyledText-sc-oo0kvp-0 bWSOET NewProductCard__ProductTitle_Desktop-sc-j0e7tu-4 cQhePS NewProductCard__ProductTitle_Desktop-sc-j0e7tu-4 cQhePS'})
    price= soup.find_all('h5',{'class':'Text__StyledText-sc-oo0kvp-0 hiHdyy'})
    disc = soup.find_all('span', {'class' : 'Text__StyledText-sc-oo0kvp-0 lnonyH'})
    name = [_.text for _ in name]
    price= [_.text for _ in price]
    disc = [_.text for _ in disc]
    # to accomodate for cases with no discount
    p = re.compile('₹0')
    i = next(i for i in range(len(price)) if p.findall(price[i]))
    disc.insert(i, '0% off')
    df = pd.DataFrame(data = {'Name':name, 'Price':price, 'Discount':disc})
    return df
    

In [7]:
def question5a(url):
    page = requests.get(url)
    soup = bs(page.content)
    team = soup.find_all('span', {'class':'u-hide-phablet'})
    details = soup.find_all('td', {'class':'table-body__cell u-center-text'})
    rating  = soup.find_all('td', {'class':'table-body__cell u-text-right rating'})

    team = [_.text for _ in team][:10]
    details = [_.text for _ in details]
    rating  = [_.text for _ in rating]

    match = [soup.find('td', {'class':'rankings-block__banner--matches'}).text]+details[0:18:2]
    point = [soup.find('td', {'class':'rankings-block__banner--points'}).text] +details[1:18:2]
    rating= [soup.find('td', {'class':'rankings-block__banner--rating u-text-right'}).text.split('\n')[1].strip()] + rating[:9]
    df = pd.DataFrame(data = {'Team':team, 'Matches':match, 'Points':point, 'Rating':rating}, index = range(1,11))
    return df

In [8]:
def question5b(url):
    page = requests.get(url)
    soup = bs(page.content)
    name = soup.find_all('td',{'class':'table-body__cell rankings-table__name name'})
    team = soup.find_all('span',{'class':'table-body__logo-text'})
    rate = soup.find_all('td',{'class':'table-body__cell rating'})

    name = soup.find('div', {'class':'rankings-block__banner--name-large'}).text.split('\n') + [_.text.split('\n')[1] for _ in name][:9]
    team = [soup.find('div',{'class':'rankings-block__banner--nationality'}).text.split('\n')[2]]+ [_.text for _ in team][:9]
    rate= [soup.find('div',{'class':'rankings-block__banner--rating'}).text]+[_.text for _ in rate][:9]
    df = pd.DataFrame(data = {'Name':name, 'Team':team, 'Rating':rate}, index = range(1,11))
    return df

In [9]:
def question5c(url):
    page = requests.get(url)
    soup = bs(page.content)
    name = soup.find_all('td',{'class':'table-body__cell rankings-table__name name'})
    team = soup.find_all('span',{'class':'table-body__logo-text'})
    rate = soup.find_all('td',{'class':'table-body__cell rating'})

    name = soup.find('div', {'class':'rankings-block__banner--name-large'}).text.split('\n') + [_.text.split('\n')[1] for _ in name][:9]
    team = [soup.find('div',{'class':'rankings-block__banner--nationality'}).text.split('\n')[2]]+ [_.text for _ in team][:9]
    rate= [soup.find('div',{'class':'rankings-block__banner--rating'}).text]+[_.text for _ in rate][:9]
    df = pd.DataFrame(data = {'Name':name, 'Team':team, 'Rating':rate}, index = range(1,11))
    return df

In [10]:
def question6a(url):
    page = requests.get(url)
    soup = bs(page.content)
    team = soup.find_all('span', {'class':'u-hide-phablet'})
    details = soup.find_all('td', {'class':'table-body__cell u-center-text'})
    rating  = soup.find_all('td', {'class':'table-body__cell u-text-right rating'})

    team = [_.text for _ in team][:10]
    details = [_.text for _ in details]
    rating  = [_.text for _ in rating]

    match = [soup.find('td', {'class':'rankings-block__banner--matches'}).text]+details[0:18:2]
    point = [soup.find('td', {'class':'rankings-block__banner--points'}).text] +details[1:18:2]
    rating= [soup.find('td', {'class':'rankings-block__banner--rating u-text-right'}).text.split('\n')[1].strip()] + rating[:9]
    df = pd.DataFrame(data = {'Team':team, 'Matches':match, 'Points':point, 'Rating':rating}, index = range(1,11))
    return df

In [11]:
def question6b(url):
    page = requests.get(url)
    soup = bs(page.content)
    name = soup.find_all('td',{'class':'table-body__cell rankings-table__name name'})
    team = soup.find_all('span',{'class':'table-body__logo-text'})
    rate = soup.find_all('td',{'class':'table-body__cell rating'})

    name = soup.find('div', {'class':'rankings-block__banner--name-large'}).text.split('\n') + [_.text.split('\n')[1] for _ in name][:9]
    team = [soup.find('div',{'class':'rankings-block__banner--nationality'}).text.split('\n')[2]]+ [_.text for _ in team][:9]
    rate= [soup.find('div',{'class':'rankings-block__banner--rating'}).text]+[_.text for _ in rate][:9]
    df = pd.DataFrame(data = {'Name':name, 'Team':team, 'Rating':rate}, index = range(1,11))
    return df

In [12]:
def question6c(url):
    page = requests.get(url)
    soup = bs(page.content)
    name = soup.find_all('td',{'class':'table-body__cell rankings-table__name name'})
    team = soup.find_all('span',{'class':'table-body__logo-text'})
    rate = soup.find_all('td',{'class':'table-body__cell rating'})

    name = soup.find('div', {'class':'rankings-block__banner--name-large'}).text.split('\n') + [_.text.split('\n')[1] for _ in name][:9]
    team = [soup.find('div',{'class':'rankings-block__banner--nationality'}).text.split('\n')[2]]+ [_.text for _ in team][:9]
    rate= [soup.find('div',{'class':'rankings-block__banner--rating'}).text]+[_.text for _ in rate][:9]
    df = pd.DataFrame(data = {'Name':name, 'Team':team, 'Rating':rate}, index = range(1,11))
    return df

In [13]:
def question7(url):
    page= requests.get(url)
    soup= bs(page.content)
    title = soup.find_all('a', {'class':'entry-title-link'})
    time  = soup.find_all('time', {'class':'entry-time'})
    content = soup.find_all('div',{'class':'entry-content'})
    link = soup.find_all('iframe', {'class':'youtube-player'})

    title   = [_.text for _ in title]
    time    = [_.text for _ in time]
    content = [_.text.replace('\n','').strip() for _ in content]
    link    = [_['src'] for _ in link]
    link.insert(4, 'No Link')
    df = pd.DataFrame(data={'Title':title,'Date':time,'Content':content,'Youtube Link':link}, index = range(1,11))
    return df

In [14]:
def question8(url):
    page = requests.get(url)
    soup = bs(page.content)
    name = soup.find_all('span',{'class':'overflow-hidden overflow-ellipsis whitespace-nowrap max-w-80pe po:max-w-full'})
    price= soup.find_all('div',{'id':'minDeposit'})
    emi  = soup.find_all('div',{'id':'roomType'})
    text = soup.find_all('span', {'class':'flex'})

    name = [_.text for _ in name]
    price= [_.find('span').text for _ in price]
    emi  = [_.text for _ in emi]
    place= [_.find('a', {'class':'text-default-color align-bottom underline hover:text-primary-color'}) for _ in text]
    place= [_['href'].split('/')[3] for _ in place if _ != None]

    df = pd.DataFrame(data = {'House Title':name, 'Location':place,
                              'EMI':emi, 'Price':price})
    return df

    

In [ ]:
def question9(url):
    '''
    Question 9 couldn't be attempted because my IP address blocked since I'm
    not in their service area. I have attached a screenshot of the same along with 
    the link the submission message
    I can't install an VPN or proxy browser because this is a work laptop
    and I wouldn't be allowed to install one.
    Thank you
    '''

In [15]:
def question10(url):
    page = requests.get(url)
    soup = bs(page.content)
    product = soup.find_all('div', {'class':'productCardDetail'})
    img     = soup.find_all('img', {'class':'productImgTag'})
    name  = [_.find('h3').text for _ in product][0:10]
    price = [_.find('span').text for _ in product][0:10]
    imgurl= [_['src'] for _ in img][0:10]
    df = pd.DataFrame(data = {'Name':name, 'Price':price, 'Img URL':imgurl}, index = range(1, 11))

